In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
            .appName('sparkSQL') \
            .getOrCreate()

In [3]:
from pyspark.sql.types import Row
from datetime import datetime

In [40]:
record = spark.sparkContext.parallelize([
                        Row(id=1,
                           name='Jill',
                           active=True,
                           clubs = ['chess', 'hockey'],
                           subjects = {'math':80, 'English':56},
                           enrolled = datetime(2014,8,1,14,1,5)
                        ),
                        Row(id=2,
                           name='George',
                           active=True,
                           clubs = ['chess', 'soccer'],
                           subjects = {'math':60, 'English':96},
                           enrolled = datetime(2015,3,21,8,2,5)
                        )]
                       )

In [8]:
record_df = record.toDF()
record_df.show()

+------+---------------+-------------------+---+------+--------------------+
|active|          clubs|           enrolled| id|  name|            subjects|
+------+---------------+-------------------+---+------+--------------------+
|  true|[chess, hockey]|2014-08-01 14:01:05|  1|  Jill|[English -> 56, m...|
|  true|[chess, soccer]|2015-03-21 08:02:05|  2|George|[English -> 96, m...|
+------+---------------+-------------------+---+------+--------------------+



In [13]:
record_df.createOrReplaceTempView('records')

In [14]:
all_records_df = sqlContext.sql('SELECT * FROM records')
all_records_df.show()

+------+---------------+-------------------+---+------+--------------------+
|active|          clubs|           enrolled| id|  name|            subjects|
+------+---------------+-------------------+---+------+--------------------+
|  true|[chess, hockey]|2014-08-01 14:01:05|  1|  Jill|[English -> 56, m...|
|  true|[chess, soccer]|2015-03-21 08:02:05|  2|George|[English -> 96, m...|
+------+---------------+-------------------+---+------+--------------------+



In [17]:
sqlContext.sql('select id, clubs[1], subjects["English"] FROM records').show()

+---+--------+-----------------+
| id|clubs[1]|subjects[English]|
+---+--------+-----------------+
|  1|  hockey|               56|
|  2|  soccer|               96|
+---+--------+-----------------+



In [18]:
record_df.createGlobalTempView('global_records')

In [19]:
sqlContext.sql('SELECT * FROM global_temp.global_records').show()

+------+---------------+-------------------+---+------+--------------------+
|active|          clubs|           enrolled| id|  name|            subjects|
+------+---------------+-------------------+---+------+--------------------+
|  true|[chess, hockey]|2014-08-01 14:01:05|  1|  Jill|[English -> 56, m...|
|  true|[chess, soccer]|2015-03-21 08:02:05|  2|George|[English -> 96, m...|
+------+---------------+-------------------+---+------+--------------------+



In [20]:
airlinesPath = '../dataset/airlines.csv'
flightsPath = '../dataset/flights.csv'
airportsPath = '../dataset/airports.csv'

In [22]:
airlines = spark.read\
                .format('csv')\
                .option('header','true')\
                .load(airlinesPath)

In [23]:
airlines.createOrReplaceTempView('airlines')

In [25]:
airlines = spark.sql('SELECT * FROM airlines')
airlines.show()

+-----+--------------------+
| Code|         Description|
+-----+--------------------+
|19031|Mackey Internatio...|
|19032|Munz Northern Air...|
|19033|Cochise Airlines ...|
|19034|Golden Gate Airli...|
|19035|  Aeromech Inc.: RZZ|
|19036|Golden West Airli...|
|19037|Puerto Rico Intl ...|
|19038|Air America Inc.:...|
|19039|Swift Aire Lines ...|
|19040|American Central ...|
|19041|Valdez Airlines: VEZ|
|19042|Southeast Alaska ...|
|19043|Altair Airlines I...|
|19044|Chitina Air Servi...|
|19045|Marco Island Airw...|
|19046|Caribbean Air Ser...|
|19047|Sundance Airlines...|
|19048|Seair Alaska Airl...|
|19049|Southeast Airline...|
|19050|Alaska Aeronautic...|
+-----+--------------------+
only showing top 20 rows



In [26]:
flights = spark.read\
                .format('csv')\
                .option('header','true')\
                .load(flightsPath)
flights.createOrReplaceTempView('flights')

In [28]:
total_distance_df = spark.sql('SELECT distance from flights')\
                    .agg({'distance':'sum'})\
                    .withColumnRenamed("sum(distance)",'total_distance')

In [31]:
all_delays_2014 = spark.sql(
                'SELECT date, airlines, flight_number, departure_delay '+
                'FROM flights where departure_delay > 0 and ' +
                'year(date) = 2014'
                )

In [32]:
all_delays_2014.orderBy(all_delays_2014.departure_delay.desc()).show(5)

+----------+--------+-------------+---------------+
|      date|airlines|flight_number|departure_delay|
+----------+--------+-------------+---------------+
|2014-04-27|   20366|         5246|          99.00|
|2014-04-27|   19393|         2948|          99.00|
|2014-04-27|   20366|         5365|          99.00|
|2014-04-26|   19977|          616|          99.00|
|2014-04-27|   20366|         6030|          99.00|
+----------+--------+-------------+---------------+
only showing top 5 rows



In [33]:
delay_per_airline = spark.sql(
                'SELECT date, airlines,departure_delay '+
                'FROM flights'
                ) \
                .groupBy('airlines')\
                .agg({'departure_delay' : 'avg'})\
                .withColumnRenamed('avg(departure_delay)', 'departure_delay')

In [34]:
delay_per_airline.show(5)

+--------+-------------------+
|airlines|    departure_delay|
+--------+-------------------+
|   19690|-2.1981308411214955|
|   19930|-0.6991515343747522|
|   20437|  5.110621095185594|
|   19393| 13.429567657134724|
|   19977|  8.818392620527979|
+--------+-------------------+
only showing top 5 rows



In [35]:
delay_per_airline.createOrReplaceTempView('delay_per_airline')

In [38]:
delay_per_airline = spark.sql(
                    'select * From delay_per_airline ' +
                    'join airlines on ' +
                    'airlines.code = delay_per_airline.airlines ' +
                    'order by departure_delay DESC'
                    )

In [39]:
products = spark.read\
                .format('csv')\
                .option('header','true')\
                .load('../dataset/products.csv')

In [43]:
import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func

In [44]:
windowSpec1 = Window.partitionBy(products['category'])\
                    .orderBy(products['price'].desc())

In [46]:
price_rank = (func.rank().over(windowSpec1))

In [48]:
product_rank = products.select(products['product'],
                              products['category'],
                              products['price']
                              ).withColumn('rank',price_rank )

In [49]:
product_rank.show()

+----------+--------+-----+----+
|   product|category|price|rank|
+----------+--------+-----+----+
|    iPhone|  Mobile|  999|   1|
|Samsung JX|  Mobile|  799|   2|
|Redmi Note|  Mobile|  399|   3|
|   OnePlus|  Mobile|  356|   4|
|        Mi|  Mobile|  299|   5|
|  Micromax|  Mobile|  249|   6|
|Samsung TX|  Tablet|  999|   1|
|      iPad|  Tablet|  789|   2|
|    Lenovo|  Tablet|  499|   3|
|        Xu|  Tablet|  267|   4|
+----------+--------+-----+----+



In [50]:
windowspec2 = Window.partitionBy(products['category'])\
                .orderBy(products['price'].desc())\
                .rowsBetween(-1,0)

In [52]:
price_max = func.max(products['price']).over(windowspec2)

In [55]:
product_w_price = products.select(products['product'],
                              products['category'],
                              products['price']
                              ).withColumn('price_max',price_max )

In [56]:
product_w_price.show()

+----------+--------+-----+---------+
|   product|category|price|price_max|
+----------+--------+-----+---------+
|    iPhone|  Mobile|  999|      999|
|Samsung JX|  Mobile|  799|      999|
|Redmi Note|  Mobile|  399|      799|
|   OnePlus|  Mobile|  356|      399|
|        Mi|  Mobile|  299|      356|
|  Micromax|  Mobile|  249|      299|
|Samsung TX|  Tablet|  999|      999|
|      iPad|  Tablet|  789|      999|
|    Lenovo|  Tablet|  499|      789|
|        Xu|  Tablet|  267|      499|
+----------+--------+-----+---------+



In [57]:
windowspec3 = Window.partitionBy(products['category'])\
                .orderBy(products['price'].desc())\
                .rangeBetween(-sys.maxsize,sys.maxsize)

In [59]:
price_diff = func.max(products['price']).over(windowspec3) - products['price']

In [60]:
product_price_diff = products.select(products['product'],
                              products['category'],
                              products['price'],
                              price_diff.alias('price_diff')
                              )

In [61]:
product_price_diff.show()

+----------+--------+-----+----------+
|   product|category|price|price_diff|
+----------+--------+-----+----------+
|    iPhone|  Mobile|  999|       0.0|
|Samsung JX|  Mobile|  799|     200.0|
|Redmi Note|  Mobile|  399|     600.0|
|   OnePlus|  Mobile|  356|     643.0|
|        Mi|  Mobile|  299|     700.0|
|  Micromax|  Mobile|  249|     750.0|
|Samsung TX|  Tablet|  999|       0.0|
|      iPad|  Tablet|  789|     210.0|
|    Lenovo|  Tablet|  499|     500.0|
|        Xu|  Tablet|  267|     732.0|
+----------+--------+-----+----------+

